<a href="https://colab.research.google.com/github/JamieBali/sudoku/blob/main/CNN_Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import os, time

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

# Data

In [ ]:
filepath = "/content/gdrive/My Drive/ColabNotebooks/IP/sudoku.csv"
inp = pd.read_csv(filepath)

puzzles = inp["quizzes"]
solutions = inp["solutions"]

In [ ]:
trivial = [[8,9,1,2,7,0,5,6,3],[6,0,3,1,8,5,9,0,0],[4,5,7,6,3,9,0,0,2],[5,0,6,4,1,7,2,3,0],[7,0,2,9,0,3,8,1,6],[3,1,0,0,2,6,0,5,4],[9,3,8,5,4,0,6,7,0],[1,6,4,0,9,0,3,2,5],[0,7,5,3,6,1,4,9,8]]

# this is a very simple 4x4 sudoku from sudoku.com
four = [[0,3,4,0],[4,0,0,2],[1,0,0,3],[0,2,1,0]]

# these three were taken randomly from sudoku.com
easy = [[0,0,0,0,7,9,0,3,0],[5,0,2,0,6,1,4,7,8],[3,7,6,0,8,5,9,0,2],[0,1,7,5,0,0,8,0,0],[2,0,9,8,3,0,0,0,0],[0,0,0,0,2,0,0,4,0],[0,0,0,0,5,0,2,0,1],[0,2,3,0,0,0,0,5,4],[1,0,0,7,0,0,0,0,0]]
medium = [[0,3,1,0,5,0,0,2,0],[0,0,0,0,0,2,9,0,5],[2,0,0,0,1,0,0,0,0],[3,5,0,0,9,0,0,7,0],[7,0,0,5,0,0,0,4,0],[0,1,0,7,0,3,2,0,0],[1,2,6,3,0,0,0,0,0],[0,9,0,8,0,5,0,0,0],[5,0,0,0,2,0,7,0,0]]
hard = [[0,4,0,0,0,5,0,6,0],[0,0,5,4,2,0,0,0,0],[0,0,1,6,0,3,5,0,4],[0,0,0,0,0,0,7,0,0],[0,3,7,0,0,0,0,1,0],[9,0,0,0,0,4,3,5,0],[0,0,4,2,5,0,0,0,0],[0,0,0,0,0,0,0,7,6],[6,0,9,0,7,0,0,0,5]]

# this is the solved version of the "easy" sudoku above.
solved = [[8,4,1,2,7,9,6,3,5],[5,9,2,3,6,1,4,7,8],[3,7,6,4,8,5,9,1,2],[4,1,7,5,9,6,8,2,3],[2,5,9,8,3,4,1,6,7],[6,3,8,1,2,7,5,4,9],[7,6,4,9,5,3,2,8,1],[9,2,3,6,1,8,7,5,4],[1,8,5,7,4,2,3,9,6]]

# Model Creation and Training

In [ ]:
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
from sklearn.model_selection import train_test_split

class sudokuSolver():

  model = None

  def __init__(self, pre_trained = 0):
    if pre_trained == 1:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_100.keras")
    elif pre_trained == 2:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_1000.keras")
    elif pre_trained == 3:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_10000.keras")
    elif pre_trained == 4:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_100000.keras")
    elif pre_trained == 5:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_500000.keras")
    elif pre_trained == 6:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/trained_on_full.keras")
    else:
      self.model = keras.models.Sequential()

      self.model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
      self.model.add(BatchNormalization())
      self.model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
      self.model.add(BatchNormalization())
      self.model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

      self.model.add(Flatten())
      self.model.add(Dense(81*9))
      self.model.add(Reshape((-1, 9)))
      self.model.add(Activation('softmax'))

      self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  
  def format_puzzle(self, puzzle):
    formatted_puzzle = []
    for x in range(9):
      row = []
      temp = puzzle[x*9:(x+1)*9]
      for y in temp:
        row.append(int(y))
      formatted_puzzle.append(row)
    formatted_puzzle = np.array(formatted_puzzle)
    return self.normalise(formatted_puzzle).reshape((9,9,1))

  def normalise(self, val):
    return (val/9) - 0.5

  def denormalise(self, val):
    return (val+0.5) * 9

  def save_model(self, name):
    self.model.save("/content/gdrive/My Drive/ColabNotebooks/IP/" + name)

  def train(self, tr_puzzles, tr_solutions, split = 0.2):
    train_puzzles = []
    train_solutions = []
    for x in tr_puzzles:
      train_puzzles.append(self.format_puzzle(x).reshape((9,9,1)))
    for x in tr_solutions:
      train_solutions.append(np.array([int(t) for t in x]).reshape((81,1))-1)

    train_puzzles, test_puzzles, train_solutions, test_solutions = train_test_split(tr_puzzles, tr_solutions, test_size=split)

    print("Training with " + str(len(train_puzzles)) + " puzzles.")

    self.model.fit(train_puzzles, train_solutions, batch_size = 64, epochs=2, verbose=1)

  def get_model(self):
    return self.model

  def solve(self, puzzle):
    solution = np.copy(puzzle).reshape((9,9))
    while True:
      formatted_puzzle = self.format_puzzle(solution)

      output = self.model.predict(formatted_puzzle).squeeze()

      predictions = np.argmax(output, axis = 1).reshape((9,9)) + 1
      probabilities = np.around(np.max(output, axis=1).reshape((9,9)), 2)

      mask = np.array(solution == 0)
      if mask.sum() == 0:
        break

      probabilities *= mask

      index_of_highest = np.argmax(probabilities)
      x, y = (index_of_highest // 9), (index_of_highest % 9)

      solution[x][y] = predictions[x][y]

    return solution

# Training the Solver

In [ ]:
model = sudokuSolver(0)
model.train(puzzles, solutions, 0.99)


Training with 10000 puzzles.
Epoch 1/2


ValueError: ignored

# Solver

In [ ]:
game = hard
with tf.device("/device:GPU:0"):
  model = sudokuSolver(6)
  print(model.solve(game))

[[2 4 3 9 1 5 8 6 7]
 [8 6 5 4 2 7 1 9 3]
 [7 9 1 6 8 3 5 2 4]
 [5 2 6 1 3 8 7 4 9]
 [4 3 7 5 9 2 6 1 8]
 [9 1 8 7 6 4 3 5 2]
 [3 7 4 2 5 6 9 8 1]
 [1 5 2 8 4 9 2 7 6]
 [6 8 9 3 7 1 4 3 5]]


In [ ]:
model.save_model("trained_on_500000.keras")